Copyright 2024 Lehi Gracia

### Save?

In [21]:
save_to_mongo = False

import subprocess, os
if save_to_mongo:
    mongod = subprocess.Popen(
        ['mongod', '--dbpath', os.path.expanduser(os.getenv('MONGODB_PATH')), '--logpath', os.path.expanduser(os.getenv('MONGODB_LOG'))]
    )

### Markets

In [22]:

import warnings
warnings.filterwarnings('ignore')

from yfinance_utils import list_utils, file_utils, constants
import yfinance 
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from finta import TA



COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
symbols = list_utils.get_markets()
names = list_utils.markets
    
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="MARKETS", title_x=0.5, width=600, height=400)

data = yfinance.download(symbols, period='1y')
gains = {}
for symbol in symbols:
    df1 = pd.DataFrame()
    df1 = data.loc[:,(slice(None),symbol)]
    df1.columns = COLUMNS
    df1['sma'] = TA.SMA(df1, period=50)
    df1 = df1[50:]
    df1.reset_index(inplace=True)

    fig.add_trace(
        go.Scatter(y=df1['Close'], x=df1['Date'], name=names[symbol]),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(y=df1['sma'], x=df1['Date'], name=f'sma'),
        secondary_y=False
    )

    temp_1 = df1['Close'].iloc[-1]
    temp_2 = df1['Close'].iloc[-30]
    gains[names[symbol]] = round(((temp_1 - temp_2) / temp_2 * 100),2)
fig.show()

print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol in symbols:
    print(f'{symbol:4}:{gains[names[symbol]]:6}: {names[symbol]}')



[*********************100%***********************]  3 of 3 completed


PERFORMANCE IN THE LAST 30 DAYS:
SPY :  -2.1: S&P 500
QQQ :   0.4: NASDAQ 100
DIA : -4.23: DOW JONES


### Implied Volaility

In [23]:
import yfinance
from finta import TA
from plotly.subplots import make_subplots
_t = yfinance.Ticker('^VIX')

COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
_d = _t.history(period='1y')
_d['sma'] = TA.SMA(_d, period=30)

_f=make_subplots(specs=[[{"secondary_y": True}]])
_f.update_layout(title=f"Market Volatility (VIX)", title_x=0.5, width=600, height=400)
_d.reset_index(inplace=True)
_f.add_trace(
    go.Scatter(y=_d['Close'], x=_d['Date'], name="VIX"),
    secondary_y=False
)
_f.add_trace(
    go.Scatter(y=_d['sma'], x=_d['Date'], name=f'sma'),
    secondary_y=False
)
_f.show()

### Sectors

In [24]:

from yfinance_utils import list_utils
from datetime import datetime

symbols = list_utils.get_sectors()
names = list_utils.sectors
COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
NUMBER_OF_TOP_PERFORMERS_IN_MARKETS = 3

fig2=make_subplots(rows=3, cols=1)
fig2.update_layout(title=f"TOP {NUMBER_OF_TOP_PERFORMERS_IN_MARKETS} SECTORS", title_x=0.5, width=800, height=600)

top = list_utils.get_top_performers(symbols, size=NUMBER_OF_TOP_PERFORMERS_IN_MARKETS)
_list = [i[0] for i in top]

data = yfinance.download(_list, period='1y')
counter=1
for symbol in _list:
    _d2 = pd.DataFrame()
    _d2 = data.loc[:,(slice(None),symbol)]
    _d2.columns = COLUMNS
    _d2['sma'] = TA.SMA(_d2, period=50)
    _d2.reset_index(inplace=True)
    fig2.add_trace(
        go.Scatter(y=_d2['Close'], x=_d2['Date'], name=names[symbol]),
        secondary_y=False,
        row=counter, col=1
    )
    fig2.add_trace(
        go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma'),
        secondary_y=False,
        row=counter, col=1
    )
    counter = counter + 1
fig2.show()

today = datetime.today().strftime('%Y-%m-%d')
print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol, value in top:
    if save_to_mongo:
        file_utils.save_to_mongo({"DATE":today, "TICK":symbol, "PERFORMANCE":value, "SECTOR":names[symbol]}, "top_sectors")



[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  3 of 3 completed


PERFORMANCE IN THE LAST 30 DAYS:


###  Getting 5 best performing stocks in each of top 3 sectors

In [25]:

from yfinance_utils import list_utils
NUMBER_OF_TOP_PERFORMERS_IN_SECTORS = 5
_sym = {}
for symbol, value in top:
    _l = []
    for s in list_utils.SECTORS[symbol].keys():
        _l.append(s)
    _tp = list_utils.get_top_performers(_l, size=NUMBER_OF_TOP_PERFORMERS_IN_SECTORS)
    _l = [i[0] for i in _tp]
    _sym[symbol] = _l
    
for i in list(_sym.keys()):
    _f=make_subplots(rows=NUMBER_OF_TOP_PERFORMERS_IN_SECTORS, cols=1)
    _f.update_layout(title=f"TOP {NUMBER_OF_TOP_PERFORMERS_IN_SECTORS} from {names[i]}", title_x=0.5, width=800, height=800)
    
    data = yfinance.download(_sym[i], period='1y')
    counter = 1
    for symbol in _sym[i]:
        _d2 = pd.DataFrame()
        _d2 = data.loc[:,(slice(None),symbol)]
        _d2.columns = COLUMNS
        _d2['sma'] = TA.SMA(_d2, period=50)
        _d2.reset_index(inplace=True)
        _f.add_trace(
            go.Scatter(y=_d2['Close'], x=_d2['Date'], name=symbol),
            secondary_y=False,
            col=1, row=counter
        )
        _f.add_trace(
            go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma'),
            secondary_y=False,
            col=1, row=counter
        )
        _f.update_yaxes(side='right')
        price = f"Date: {today}, Open:{round(_d2['Open'].iloc[-1],2)}, High:{round(_d2['High'].iloc[-1],2)}, Low:{round(_d2['Low'].iloc[-1],2)}, Close: {round(_d2['Close'].iloc[-1],2)}"
        _f.add_annotation(xref='x domain',
                        yref='y domain',
                        x=0.01,
                        y=1,
                        text=price, 
                        showarrow=False,
                        font=dict(size=10),
                        row=counter, col=1)
        counter = counter + 1
    _f.show()

if save_to_mongo:
    _sym['DATE'] = datetime.today().strftime('%Y-%m-%d')
    file_utils.save_to_mongo(_sym, "top_performers")

[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  69 of 69 completed
[*********************100%***********************]  21 of 21 completed
[*********************100%***********************]  5 of 5 completed


[*********************100%***********************]  5 of 5 completed


[*********************100%***********************]  5 of 5 completed


### Top Analysts Ratings

In [4]:

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
pd.set_option('display.max_colwidth', None)



client = MongoClient('mongodb://localhost:27017/')
db = client['bigdata']
collection = db.market

yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
today = datetime.today().strftime('%Y-%m-%d')


res = collection.find(
                 {'DATE':today, 'script':'daily_analysts_ratings'}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2).sort_values(by="TOTAL", axis=0, ascending=False).iloc[:20]
if data.empty:
    print('No Data Today')
else:
    print(data)


          DATE  TICK  UP  DOWN  HOLD    AVG  TOTAL
20  2025-01-15  META  58     2     8  85.29     68
52  2025-01-15  AMZN  64     1     3  94.12     68
1   2025-01-15  NVDA  59     0     4  93.65     63
25  2025-01-15  MSFT  53     0     5  91.38     58
31  2025-01-15  DDOG  40     0     6  86.96     46
2   2025-01-15  AVGO  38     0     5  88.37     43
34  2025-01-15   WMT  37     1     3  90.24     41
35  2025-01-15  MRVL  33     1     2  91.67     36
48  2025-01-15   BSX  31     0     4  88.57     35
28  2025-01-15   COP  25     0     3  89.29     28
4   2025-01-15   UNH  25     0     2  92.59     27
45  2025-01-15   UAL  23     1     0  95.83     24
6   2025-01-15   HWM  21     1     2  87.50     24
51  2025-01-15  VICI  21     0     3  87.50     24
21  2025-01-15   DAL  22     1     0  95.65     23
7   2025-01-15  SPGI  21     0     2  91.30     23
41  2025-01-15  TRGP  20     1     2  86.96     23
12  2025-01-15  SNPS  20     0     3  86.96     23
5   2025-01-15   LYV  19     2 

### Gap up close high

In [5]:

res = collection.find(
                 {'DATE':today, 'script':{'$in':['daily_gap_up_close_high']}}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
if data.empty:
    print('No Data Today')
else:
    print(data)

No Data Today


### Price drops and jumps

In [9]:

res = collection.find(
                 {'DATE':today, 'script':{'$in':['daily_jump_price_down','daily_jump_price_up']}}, 
                 {'_id':0, 'script':0, 'timestamp':0, 'VOLUME':0  }
                 )

data = pd.DataFrame(res).round(2)
if data.empty:
    print('No Data Today')
else:
    print(data)

No Data Today


### Turn off mongo

In [10]:
print('The END')

The END
